# Instalação das bibliotecas necessarias

In [1]:
#!pip install -r requirements.txt
#!pip install --upgrade keras

# Dataset

In [92]:
# O dataset utilizado trata se duas categorias de imagens sendo elas boca aberta e boca fechada.
# Organizadas seguindo a seguinte estrutura:

"""
mouth_dataset
    train
        open
        closed
    valid
        open
        closed
    test
        open
        closed
"""
# Como a base de dados obtida continha poucas imagens para o treinamento foi necessário o manipulação nas imagens para aumentar a base de dados e melhor a identificação de features.
# As imagens utilizadas no treinamento foram retiradas das bases de imagens do google, duckduckgo e imgur.

'\nmouth_dataset\n    train\n        open\n        closed\n    valid\n        open\n        closed\n    test\n        open\n        closed\n'

In [95]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

import os

def processing_images(train_datagen):
    for root, dirs, files in os.walk("./boca/train/aberta"):
        for file in files:
            img = load_img(os.path.join(root, file))
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            i = 0
            for batch in train_datagen.flow(x, batch_size=1,
                                      save_to_dir='preview', save_prefix='aberta', save_format='jpeg'):
                i += 1
                if i > 20:
                    break
                    
# processing_images(train_datagen)

# Definindo arquitetura da rede neural convolucional

In [96]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [97]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Tratando as imagens e aumentando a base de dados

In [98]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32

training_set = train_datagen.flow_from_directory('mouth_dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_set = validation_datagen.flow_from_directory('mouth_dataset/valid',
                                            target_size = (64, 64),
                                            batch_size = batch_size,
                                            class_mode = 'binary')


Found 1427 images belonging to 2 classes.
Found 29 images belonging to 2 classes.


In [101]:
%%time
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit_generator(training_set,
                         steps_per_epoch = 2000 // batch_size,
                         epochs=50,
                         validation_data = validation_set,
                         validation_steps = 800 // batch_size,
                         callbacks=callbacks_list)


Epoch 1/1
200/200 [==============================] - 228s 1s/step - loss: 0.3239 - acc: 0.8734 - val_loss: 0.9496 - val_acc: 0.7931

Epoch 00001: val_acc improved from -inf to 0.79310, saving model to model_weights.h5
CPU times: user 5min 42s, sys: 1.93 s, total: 5min 43s
Wall time: 3min 51s


In [48]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


# Teste manual

In [108]:
from keras.models import load_model
arroz = load_model('model_weights.h5')

import numpy as np
from keras.preprocessing import image


In [109]:
test_image = image.load_img('mouth_dataset/test/ab.jpg', target_size = (64, 64, 3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = arroz.predict(test_image)
training_set.class_indices
if result[0][0] != 1:
    prediction = 'open'
else:
    prediction = 'closed'

In [110]:
print(training_set.class_indices)
print(result[0][0], prediction)

{'aberta': 0, 'fechada': 1}
0.0 open
